<a href="https://colab.research.google.com/github/Justin21523/edge-deid-studio/blob/feature%2Fadd-gpt2-finetune-notebook/notebooks/03_finetune_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['HF_HOME'] = '/content/drive/MyDrive/hf_cache'
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/hf_cache/transformers'
os.makedirs(os.environ['TRANSFORMERS_CACHE'], exist_ok=True)


In [ ]:
from huggingface_hub import login
from getpass import getpass

hf_token = getpass("請貼上你的 Hugging Face token：")
login(token=hf_token)


In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())


邏輯說明
這幾個 cell 與 02_train_ner.ipynb 如出一轍：第一步把 Transformers、Datasets、Accelerate 安裝好，第二步把快取搬到 Drive，第三步登入 Hugging Face（為了後續上傳模型或下載 private model），最後確認是否偵測到 GPU。雖然我們是先在免費版 Colab 上編輯，但請確保程式碼無誤，一旦切到 Pro 或有 GPU 時，就能直接跑訓練。

## 二、下載與檢視資料集（Cell 5）

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("wikitext", "wikitext-2-raw-v1")
print(raw_datasets)

**做了什麼**
使用 `load_dataset("wikitext", "wikitext-2-raw-v1")` 下載 WikiText-2 原始文字資料集，這是一個常用的小型語言模型微調語料。可以看到 train、validation、test 三個 split。

## 三、Tokenizer 與 Model 載入（Cell 6）

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_checkpoint = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

**為什麼這麼做**
`gpt2` 預訓練模型本身就已經是 causal LM（自回歸），我們只要把 tokenizer 與 model 載下來，後續再把語料接在後面繼續調整權重。


## 四、分段 Tokenize（Cell 7–8）

In [ ]:
def group_texts(examples):
    # 把所有行合併成一個長字串，再依 block_size 切段落
    concatenated = tokenizer.tokenize(" ".join(examples["text"]))
    total_length = len(concatenated)
    # 每段長度＝block_size 的倍數
    block_size = 128
    total_length = (total_length // block_size) * block_size
    result = {
        "input_ids": [concatenated[i : i + block_size] for i in range(0, total_length, block_size)],
        "attention_mask": [[1] * block_size] * (total_length // block_size),
    }
    return result

tokenized_datasets = raw_datasets.map(
    lambda examples: tokenizer(examples["text"]),
    batched=True,
    remove_columns=["text"]
).map(
    group_texts,
    batched=True,
)

**背後邏輯**
因為 GPT-2 是自回歸模型，需要固定長度的句子段落（block）來預測下一個 token。這裡先用 `tokenizer` 將每一句切成 token id，接著把所有 token 拼接後，依照 `block_size`（128）切成多個 chunk。每個 chunk 都會是 model 的一個訓練樣本。

## 五、DataCollator 與 Trainer 設定（Cell 9–11）

In [ ]:
python
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False  # 因為 GPT-2 是自回歸 (causal LM)，不用做遮罩填空
)

training_args = TrainingArguments(
    output_dir="models/gpt2/v1.0",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=200,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

**說明**

* **DataCollatorForLanguageModeling**：自回歸時，不要遮罩 (mlm=False)。
* **TrainingArguments**：設定 epoch、batch size、learning rate、權重衰減等等。
* **Trainer**：把 model、dataset、collator、args 統整，之後只要呼叫 `trainer.train()` 就會跑微調。

## 六、啟動訓練（Cell 12）

In [ ]:
# 切到 GPU runtime 後才執行
# trainer.train()
# trainer.save_model()

這裡我們先把 `trainer.train()` 和 `trainer.save_model()` 註解掉，確保整段程式碼可以先跑過 syntax check。等到你切到有 GPU 的環境（或 Colab Pro）時，再把註解移除，一次跑完微調並儲存模型到 `models/gpt2/v1.0`。


### 小結

目前 `03_finetune_gpt2.ipynb` 的流程已經完整：

1. 安裝並設定環境
2. 下載 WikiText-2 語料
3. 載入 GPT-2 tokenizer & model
4. 將文字切成固定長度的 block
5. 用 `Trainer` 設定 fine-tune 超參
6. （待 GPU）呼叫訓練並儲存